This notebook contains a demonstration of the fulle procedure to use a grasping service to execute a grap on a loaded ycb object. 
The procedure is the following:

1.) 

To successfully execute this notebook, a roscore needs to be running on the same machine as the notebook server and the grasping service needs to be running.
The grasping services from the `grasping-benchmark-panda` repo can be easily started using the Docker containers provided in the `grasping-benchmark-panda` repo.

In [1]:
from pathlib import Path
import logging
import importlib
import sys

from scipy.spatial.transform import Rotation as R
import numpy as np

from alr_sim.core import Scene

from alr_sim_tools.ycb_utils import YCBLoader
from alr_sim_tools.scene_utils import (
    execute_grasping_sequence,
    record_camera_data,
    reset_scene,
)
from alr_sim_tools.ros_utils import create_grasp_planner_request

sys.path.append(str(Path.cwd().parent / "alr_sim_tools" / "ros_msg_srv_definitions"))
from alr_sim_tools.ros_msg_srv_definitions.grasping_benchmarks_ros.srv import (
    GraspPlannerResponse,
    GraspPlanner,
)

import rospy

pybullet build time: May  2 2023 05:55:04


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
FACTORY_STRING = "mj_beta"
OBJECT_POS = (0.5, 0.0, 0.2)
CAM_POS = (0.5, 0.0, 1)
CAM_QUAT = [0.7071067811865476, 0.0, 0.0, -0.7071067811865475]
CAM_HEIGHT = 480
CAM_WIDTH = 640
INITIAL_ROBOT_POS = (0.0, 0.5, 0.2)
INITIAL_ROBOT_QUAT = (0, 1, 0, 0)
RENDER_MODE = Scene.RenderMode.HUMAN    
DATA_RECORDING_WAIT_TIME = 0.5
DATA_RECORDING_MOVE_TIME = 2

GRASP_PLANNER_SERVICE_ID = "contact_bench/contact_grasp_planner_service"

YCB_FOLDER = Path.home() / "Documents" / "SF-ObjectDataset" / "YCB"
YCB_OBJECT_ID = "011_banana"

In [3]:
ycb_loader = YCBLoader(ycb_base_folder=YCB_FOLDER, factory_string="mj_beta")

sim_obj = ycb_loader.get_ycb_object(
    pos=(0.5, 0.0, 0.2), quat=(0, 1, 0, 0), obj_id=YCB_OBJECT_ID, name=YCB_OBJECT_ID
)

In [4]:
camera_data, scene, agent = record_camera_data(
    factory_string=FACTORY_STRING,
    cam_pos=CAM_POS,
    cam_quat=CAM_QUAT,
    cam_height=CAM_HEIGHT,
    cam_width=CAM_WIDTH,
    robot_pos=INITIAL_ROBOT_POS,
    robot_quat=INITIAL_ROBOT_QUAT,
    object_list=[sim_obj],
    target_obj_name=YCB_OBJECT_ID,
    render_mode=RENDER_MODE,
    wait_time=DATA_RECORDING_WAIT_TIME,
    move_duration=DATA_RECORDING_MOVE_TIME,
)

In [5]:
rospy.init_node("grasp_planner_client")
importlib.reload(logging)
logging.basicConfig(level=logging.INFO)

In [6]:
grasp_req = create_grasp_planner_request(
    rgb_img=camera_data["rgb_img"],
    depth_img=camera_data["depth_img"],
    seg_img=camera_data["seg_img"],
    pc_points=camera_data["point_cloud_points"],
    pc_colors=camera_data["point_cloud_colors"],
    cam_pos=camera_data["cam_pos"],
    cam_quat=camera_data["cam_quat"],
    cam_intrinsics=camera_data["cam_intrinsics"],
    cam_height=CAM_HEIGHT,
    cam_width=CAM_WIDTH,
    num_of_candidates=1,
)

In [7]:
rospy.wait_for_service(GRASP_PLANNER_SERVICE_ID, timeout=30.0)
grasp_planner = rospy.ServiceProxy(GRASP_PLANNER_SERVICE_ID, GraspPlanner)

reply: GraspPlannerResponse = grasp_planner(grasp_req)

ServiceException: service [/contact_bench/contact_grasp_planner_service] responded with an error: b"error processing request: No module named 'tf'"

In [ ]:
best_grasp = sorted(reply.grasp_candidates, key=lambda x: x.score.data)[-1]

grasp_pos = best_grasp.pose.pose.position
grasp_pos = np.array((grasp_pos.x, grasp_pos.y, grasp_pos.z))

grasp_quat = best_grasp.pose.pose.orientation
grasp_quat = np.array((grasp_quat.w, grasp_quat.x, grasp_quat.y, grasp_quat.z))

grasp_rot = R.from_quat(grasp_quat[[1,2,3,0]]).as_matrix()

In [ ]:
print(grasp_rot)

In [ ]:
# hover_pos = grasp_pos + np.array([0, 0, 0.2])
# agent.gotoCartPositionAndQuat(hover_pos, grasp_quat)
# agent.wait(100)

In [ ]:

execute_grasping_sequence(
    agent = agent,
    grasp_pos = grasp_pos,
    grasp_quat = grasp_quat
)

In [ ]:
reset_scene(FACTORY_STRING, scene, agent)